In [27]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
from dask import delayed
from dask.multiprocessing import get
import numpy as np


In [33]:
n = 1000000

data = pd.DataFrame(np.random.rand(4*n).reshape((n, 4)), columns = ['a', 'b', 'c', 'd'])
ddata = dd.from_pandas(data, npartitions=12)

In [34]:
def fun(row):
    return np.sum(row)

res = ddata.map_partitions(lambda df: df.apply(lambda row: fun(row), axis = 1)).compute(scheduler="threads")
res

0         2.030168
1         2.305121
2         2.140922
3         1.885470
4         2.362767
            ...   
999995    2.240480
999996    2.104046
999997    2.773158
999998    1.867770
999999    2.125504
Length: 1000000, dtype: float64

# Text preprocessing

In [35]:
#import some libraries

from datatable import by, dt, f
import datatable, os, pandas as pd, numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%cd ..


/home/mluser/projects/Keywork-Extraction-Kaggle-Stackoverflow


In [36]:
data_r = datatable.fread("./input/data_no_dupes_top50.csv", max_nrows=10)
data = data_r[:, f[:].remove([f["#tags"], f["count_duples"], f[ "Tags"]])]

data.head(2)

,Title,Body,c#,java,php,javascript,android,jquery,c++,python,…,asp.net-mvc-3,image,html5,networking,homework
,▪▪▪▪,▪▪▪▪,▪,▪,▪,▪,▪,▪,▪,▪,,▪,▪,▪,▪,▪
0,Implementing Boundary Value Analysis of Softw…,<pre><code>#include&lt;iostream&gt; #include&lt;st…,0,0,0,0,0,0,1,0,…,0,0,0,0,0
1,Dynamic Datagrid Binding in Silverlight?,<p>I should do binding for datagrid dynamically at…,1,0,0,0,0,0,0,0,…,0,0,0,0,0


In [41]:
X = data[:, ["Title", "Body"]].to_pandas()
Y = data[:, data.names[2:]]

print("X --> ", X.columns)
Y.head(1)

X -->  Index(['Title', 'Body'], dtype='object')


,c#,java,php,javascript,android,jquery,c++,python,iphone,asp.net,…,asp.net-mvc-3,image,html5,networking,homework
,▪,▪,▪,▪,▪,▪,▪,▪,▪,▪,,▪,▪,▪,▪,▪
0,0,0,0,0,0,0,1,0,0,0,…,0,0,0,0,0


## Logic below

In [47]:
row = X.iloc[0]
X.head(1)

,Title,Body
0,Implementing Boundary Value Analysis of S...,<pre><code>#include&lt;iostream&gt;\n#include&...


In [51]:
row = X.iloc[0]
row

Title         Implementing Boundary Value Analysis of S...
Body     <pre><code>#include&lt;iostream&gt;\n#include&...
Name: 0, dtype: object

Steps - 
* Sample
* Remove code snippets
* Remove special characters from remaining text
* Remove stop words
* remove HTML tags
* convert to lowercase
* Stem the words using snowball stemmer

In [73]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from nltk.tokenize import word_tokenize # to break sentences into words

def striphtml(data):
    # find the html tags and remove them
    patt = re.compile('<.*?>')
    cleantxt = re.sub(patt, ' ', str(data))
    return cleantxt

def process_row(row):
    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer("english")

    questions_with_code =  0
    len_before = 0
    len_after = 0

    is_code = 0
    title, question = row[0], row[1]

    if "<code>" in question:
        questions_with_code += 1
        is_code = 1

    x = len(question) + len(title)
    len_before += x

    # Find the code piece of question
    code = str(re.findall(r'<code>(.*?)</code>', question, flags=re.DOTALL))
    # remove the code
    question = re.sub('<code>(.*?)</code>', '', question, flags=re.MULTILINE|re.DOTALL)

    title=title.encode('utf-8')

    # combine question and title, remove special characters, convert to lowercase
    queti = str(title) + str(question)
    queti = re.sub(r'[^A-Za-z]+',' ',queti).lower()

    # get words
    words = word_tokenize(str(queti))

    #Removing all single letter and and stopwords from question except for the letter 'c'
    queti_stop = ' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))

    len_after += len(queti_stop)

    return queti_stop, len(queti_stop), is_code, code, x


In [64]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
from dask import delayed
from dask.multiprocessing import get
import numpy as np


In [66]:
n = 1000000

data = X
ddata = dd.from_pandas(data, npartitions=12)

In [76]:
res = ddata.map_partitions(lambda df: df.apply(lambda row: process_row(row), axis = 1, result_type="expand")).compute(scheduler="threads")
res.columns = ['queti_stop', 'len_queti_stop', 'is_code', 'code', 'len_pre']
res

,queti_stop,len_queti_stop,is_code,code,len_pre
0,implement boundari valu analysi softwar test c...,161,1,['#include&lt;iostream&gt;\n#include&lt;stdlib...,2116
1,dynam datagrid bind silverlight bind datagrid ...,192,1,['MyClass myInstance = new MyClass();\ndataGri...,904
2,dynam datagrid bind silverlight bind datagrid ...,192,1,['MyClass myInstance = new MyClass();\ndataGri...,904
3,java sql sqlexcept microsoft odbc driver manag...,121,1,"['try {\n Class.forName(""sun.jdbc.odbc.Jdbc...",1056
4,better way updat feed fb php sdk novic faceboo...,179,1,"['$data = array(""message"" =&gt; ""Hello World!""...",607
5,btnadd click event open two window record ad o...,190,1,[' //calling Search.aspx on button click...,1359
6,sql inject issu prevent correct form submiss p...,539,1,"['{$variables}', '&lt;div width=""100%""&gt;\n{$...",4874
7,undefin symbol architectur objc class skpsmtpm...,456,1,['Undefined symbols for architecture i386:\n\n...,1279
8,java lang nosuchmethoderror javax servlet serv...,908,1,"['&lt;?xml version=""1.0"" encoding=""UTF-8""?&gt;...",1852
9,obtain updat locat use gps servic app two butt...,355,1,['public void startButton(View view) {\n\n ...,1053
